# Preprocessing Data

In [2]:
import os
import music21
import json
import tensorflow.keras
import keras.utils
import numpy as np

# us = environment.UserSettings()
# us.create()
# us['musescoreDirectPNGPath'] = 'Applications/'
# environment.set('musescoreDirectPNGPath', "./")
# music21.converter.program.musescoreDirectPNGPath =  "./"

2024-05-20 08:06:44.409310: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Setup

DATASET_PATH = "deutschl/erk"
ACCEPTABLE_DURATIONS = [.25, .5, .75, 1, 1.5, 2, 3, 4]
SAVE_DIR = "./convertedFile"
SINGLE_FLE_DATASET = "./singleFileDataset"
MAPPING = {}
SEQUENCE_LENGTH = 64

In [4]:
def preprocess(dataset_pass):
    # load
    songs = loadSongs(dataset_pass)
    print(f"Number of loaded songs :{len(songs)}")

    for index, song in enumerate(songs): 
        # filter files by duration 
        if not hasAcceptableDuration(song, ACCEPTABLE_DURATIONS):
            continue

        # transpose to smae scale
        song = transpose(song)

        # time series representation encoding 
        encodedSong = encode(song)

        # save 
        save_path = os.path.join(SAVE_DIR, str(index))
        with open(save_path, "w") as fp: 
            fp.write(encodedSong)

def transpose(song):
    # key
    parts = song.getElementsByClass(music21.stream.Part)
    measure_part_zero = parts[0].getElementsByClass(music21.stream.Measure)
    key = measure_part_zero[0][4]

    if not isinstance(key, music21.key.Key):
        key = song.analyze("key")

    # interval transposition
    if key.mode == 'major' : 
        interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch("C"))
    else:
        interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch("A"))
    
    return song.transpose(interval)
    

def loadSongs(dataset_pass):
    songs = []
    for path, subdir, files in os.walk(dataset_pass):
        for file in files: 
            if file[-3:] == "krn":
                song = music21.converter.parse(os.path.join(path, file))
                songs.append(song)
    return songs

def hasAcceptableDuration(song, acceptables):
    for singleNote in song.flatten().notesAndRests:
        if singleNote.duration.quarterLength not in acceptables:
            return False

    return True

def encode(song, timestep=.25):
    encodedSong = []
    for event in song.flatten().notesAndRests:
        if isinstance(event, music21.note.Note):
            symbol = event.pitch.midi
            
        elif isinstance(event, music21.note.Rest):
            symbol = "r"

        # convert into timeseries notation
        steps = int(event.duration.quarterLength/timestep)
        for step in range(steps):
            if step == 0 :
                encodedSong.append(symbol)
            else: 
                encodedSong.append("_")
                
    encodedSong = " ".join(map(str, encodedSong))

    return encodedSong
    

In [5]:
def createSingleFileDataset(datasetPath, fileDatasetPath):
    newSongDelimiter = "/ " * SEQUENCE_LENGTH
    songs = ""

    for path, _, files in os.walk(datasetPath):
        for file in files:
            with open(os.path.join(path, file), "r") as fp: 
                song = fp.read()
            songs = songs + song + " " + newSongDelimiter

    songs = songs[:-1]

    with open(fileDatasetPath, "w") as fp:
        fp.write(songs)

    return songs

def createMapping(songs):
    # mapping = {}
    for index, symbol in enumerate(list(set(songs.split()))):
        MAPPING[symbol] = index
    
    # with open(MAPPING_PATH, "w") as fp: 
    #     json.dump(mapping, fp)

In [6]:
def songsToIntConvertor(songs):
    intSongs = []
    songs = songs.split()
    for symbol in songs : 
        intSongs.append(MAPPING[symbol])

    return intSongs

def generatTrainingSequence(intSongs):
    inputs = []
    targets = []
    
    numberOfSequences = len(intSongs) - SEQUENCE_LENGTH
    for i in range(numberOfSequences):
        inputs.append(intSongs[i:i+SEQUENCE_LENGTH])
        targets.append(intSongs[i+SEQUENCE_LENGTH])

    # one-hot encode based on the MAPPING
    vocabSize = len(set(intSongs))
    inputs = keras.utils.to_categorical(inputs, num_classes=vocabSize)
    targets = np.array(targets)

    return inputs, targets

In [8]:
if __name__ == "__main__":
    preprocess(DATASET_PATH)
    songs = createSingleFileDataset(SAVE_DIR, SINGLE_FLE_DATASET)
    createMapping(songs)
    intSongs = songsToIntConvertor(songs)
    inputs, targets = generatTrainingSequence(intSongs)

Number of loaded songs :1700


# Model : Building and Creating

In [ ]:
# Setup
OUTPUT_UNITS = len(MAPPING)
NUM_UNITS = [256]
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
EPOCHS = 40

In [ ]:
def training(inputs, targets, outputUnits=OUTPUT_UNITS, numUnits=NUM_UNITS, loss=LOSS, learningRate=LEARNING_RATE, epochs=EPOCHS):
    # build model 
    input = keras.layers.Input(shape=(None, outputUnits))
    x = keras.layers.LSTM(numUnits[0])(input)
    x = keras.layers.Dropout(0.2)(x)

    output = keras.layers.Dense(outputUnits, activation="softmax")(x)

    model = keras.Model(input, output)
    model.compile(loss= loss, 
                  optimizer=keras.optimizers.Adam(learning_rate=learningRate),
                  metrics=['accuracy'])
    model.summary()

    model.fit(inputs, targets, epochs=epochs, batch_size=64)

    model.save("trained_model.h5")

In [24]:
if __name__ == "__main__":
    training(inputs, targets)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, None, 38)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 256)            │       302,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 38)             │         9,766 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 311,846 (1.19 MB)

 Trainable params: 311,846 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
1550/5660 ━━━━━━━━━━━━━━━━━━━━ 18:54 276ms/step - accuracy: 0.7447 - loss: 1.0419

KeyboardInterrupt: 